In [5]:
%%writefile practice75.cu
// Practice 7 Доп. задание 3
// 3. Исследуйте влияние размера блока на производительность.

#include <iostream>               // Для стандартного ввода-вывода
#include <cuda_runtime.h>         // Для CUDA Runtime API
#include <chrono>                 // Для измерения времени

using namespace std;              // Чтобы не писать std::

// CUDA KERNEL: Редукция
__global__ void reductionKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int sdata[];           // Разделяемая память блока (shared memory)
    int tid = threadIdx.x;                    // Локальный индекс потока внутри блока
    int i = blockIdx.x * blockDim.x + tid;   // Глобальный индекс элемента массива

    // Загружаем элементы в shared memory
    sdata[tid] = (i < n) ? d_input[i] : 0;   // Если индекс не выходит за массив, копируем элемент; иначе 0
    __syncthreads();                          // Синхронизация потоков внутри блока

    // Редукция внутри блока
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) { // Делим блок на пары элементов
        if (tid < s) sdata[tid] += sdata[tid + s];        // Складываем элементы попарно
        __syncthreads();                                  // Синхронизация после каждого шага
    }

    // Записываем сумму блока в глобальную память
    if (tid == 0) d_output[blockIdx.x] = sdata[0];        // Первый поток блока сохраняет результат
}

// CUDA KERNEL: Префиксная сумма
__global__ void prefixSumKernel(int *d_input, int *d_output, int n) {
    extern __shared__ int temp[];             // Shared memory для сканирования
    int tid = threadIdx.x;                    // Локальный индекс потока
    int i = blockIdx.x * blockDim.x + tid;   // Глобальный индекс элемента

    // Копируем данные в shared memory
    temp[tid] = (i < n) ? d_input[i] : 0;    // Элементы блока или 0, если вышли за предел массива
    __syncthreads();                          // Синхронизация потоков блока

    // Шаг вверх (upsweep) для построения дерева сумм
    for (int offset = 1; offset < blockDim.x; offset *= 2) {
        int index = (tid + 1) * offset * 2 - 1;       // Индекс для сложения
        if (index < blockDim.x) temp[index] += temp[index - offset]; // Складываем элементы
        __syncthreads();                               // Синхронизация перед следующим шагом
    }

    // Шаг вниз (downsweep) для вычисления префиксных сумм
    if (tid == 0) temp[blockDim.x - 1] = 0;           // Обнуляем последний элемент
    __syncthreads();

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        int index = (tid + 1) * offset * 2 - 1;       // Индекс для перестановки и суммирования
        if (index < blockDim.x) {
            int t = temp[index - offset];             // Временная переменная
            temp[index - offset] = temp[index];       // Перемещаем элемент
            temp[index] += t;                         // Добавляем временное значение
        }
        __syncthreads();                              // Синхронизация после шага
    }

    // Записываем результат в глобальную память
    if (i < n) d_output[i] = temp[tid];
}

// ФУНКЦИЯ ИЗМЕРЕНИЯ ВРЕМЕНИ
void runBlockSizeTest(int N, int threadsPerBlock) {
    cout << "\nРазмер массива: " << N << ", Потоки на блок: " << threadsPerBlock << endl; // Вывод текущего теста

    int *h_array = new int[N];                     // Массив на CPU
    for (int i = 0; i < N; i++) h_array[i] = 1;   // Заполняем единицами

    // CPU редукция
    int cpu_sum = 0;                               // Переменная для суммы
    auto cpu_start = chrono::high_resolution_clock::now(); // Начало таймера
    for (int i = 0; i < N; i++) cpu_sum += h_array[i];     // Суммируем элементы
    auto cpu_end = chrono::high_resolution_clock::now();   // Конец таймера
    chrono::duration<double> cpu_time = cpu_end - cpu_start; // Время выполнения CPU

    // GPU редукция
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock; // Количество блоков
    int *d_input, *d_intermediate;             // Указатели на GPU
    cudaMalloc(&d_input, N * sizeof(int));     // Выделяем память на GPU для входного массива
    cudaMemcpy(d_input, h_array, N * sizeof(int), cudaMemcpyHostToDevice); // Копируем данные CPU -> GPU
    cudaMalloc(&d_intermediate, blocksPerGrid * sizeof(int));            // Память под промежуточные суммы

    cudaEvent_t gpu_start, gpu_stop;           // CUDA события для измерения времени
    cudaEventCreate(&gpu_start);               // Создаем событие старта
    cudaEventCreate(&gpu_stop);                // Создаем событие окончание
    cudaEventRecord(gpu_start);                // Запуск GPU таймера

    reductionKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_intermediate, N); // Запуск ядра

    int *h_intermediate = new int[blocksPerGrid]; // Массив для сумм блоков
    cudaMemcpy(h_intermediate, d_intermediate, blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost); // GPU -> CPU
    int gpu_sum = 0;                             // Итоговая сумма
    for (int i = 0; i < blocksPerGrid; i++) gpu_sum += h_intermediate[i]; // Суммируем блоки

    cudaEventRecord(gpu_stop);                  // Остановить таймер GPU
    cudaEventSynchronize(gpu_stop);             // Ждем окончания события
    float gpu_time = 0;                         // Переменная для хранения времени GPU
    cudaEventElapsedTime(&gpu_time, gpu_start, gpu_stop); // Время GPU

    // CPU сканирование
    int *h_scan_cpu = new int[N];               // Массив для CPU сканирования
    auto cpu_scan_start = chrono::high_resolution_clock::now();           // Старт
    h_scan_cpu[0] = h_array[0];                 // Первый элемент
    for (int i = 1; i < N; i++) h_scan_cpu[i] = h_scan_cpu[i - 1] + h_array[i]; // Последовательный скан
    auto cpu_scan_end = chrono::high_resolution_clock::now();                   // Конец
    chrono::duration<double> cpu_scan_time = cpu_scan_end - cpu_scan_start;     // Продолжительность

    // GPU сканирование
    int *d_output;                              // Массив для GPU сканирования
    cudaMalloc(&d_output, N * sizeof(int));
    cudaEventRecord(gpu_start);
    prefixSumKernel<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, N); // GPU скан
    cudaEventRecord(gpu_stop);
    cudaEventSynchronize(gpu_stop);
    float gpu_scan_time = 0;
    cudaEventElapsedTime(&gpu_scan_time, gpu_start, gpu_stop);

    // Вывод времени
    cout << "CPU время (редукция): " << cpu_time.count() * 1000 << " мс" << endl;
    cout << "GPU время (редукция): " << gpu_time << " мс" << endl;
    cout << "CPU время (сканирование): " << cpu_scan_time.count() * 1000 << " мс" << endl;
    cout << "GPU время (сканирование): " << gpu_scan_time << " мс" << endl;

    // Освобождение памяти
    delete[] h_array;
    delete[] h_intermediate;
    delete[] h_scan_cpu;
    cudaFree(d_input);
    cudaFree(d_intermediate);
    cudaFree(d_output);
}

// Основная функция
int main() {
    int blockSizes[] = {128, 256, 512, 1024};   // Массив с размерами блоков для эксперимента
    int arraySizes[] = {1024, 1000000, 10000000}; // Размеры массивов

    for (int i = 0; i < 3; i++) {               // Проходим по всем массивам
        for (int j = 0; j < 4; j++) {           // Проходим по всем размерам блока
            runBlockSizeTest(arraySizes[i], blockSizes[j]); // Запуск теста
        }
    }
    return 0;                                  // Завершение
}


Overwriting practice75.cu


In [6]:
# Компиляция
!nvcc practice75.cu -o practice75 -arch=sm_75 -std=c++11            # -arch=sm_75  - архитектура GPU (Tesla T4 в Colab = sm_75)
                                                                    # -std=c++11 — стандарт C++
# Запуск
!./practice75


Размер массива: 1024, Потоки на блок: 128
CPU время (редукция): 0.002701 мс
GPU время (редукция): 0.125952 мс
CPU время (сканирование): 0.00302 мс
GPU время (сканирование): 0.023776 мс

Размер массива: 1024, Потоки на блок: 256
CPU время (редукция): 0.002839 мс
GPU время (редукция): 0.024416 мс
CPU время (сканирование): 0.003061 мс
GPU время (сканирование): 0.013408 мс

Размер массива: 1024, Потоки на блок: 512
CPU время (редукция): 0.002789 мс
GPU время (редукция): 0.022304 мс
CPU время (сканирование): 0.003008 мс
GPU время (сканирование): 0.015136 мс

Размер массива: 1024, Потоки на блок: 1024
CPU время (редукция): 0.002812 мс
GPU время (редукция): 0.029472 мс
CPU время (сканирование): 0.003002 мс
GPU время (сканирование): 0.021408 мс

Размер массива: 1000000, Потоки на блок: 128
CPU время (редукция): 2.5297 мс
GPU время (редукция): 0.163264 мс
CPU время (сканирование): 4.56379 мс
GPU время (сканирование): 0.294176 мс

Размер массива: 1000000, Потоки на блок: 256
CPU время (редукция